In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
patients = pd.read_csv('clevelanda.csv')

In [9]:
patients.head()

,age,gender,cp,trestbps,chol,fps,restecg,thalach,exang,oldpeak,slope,ca,thal,class
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [10]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
age         303 non-null int64
gender      303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
fps         303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
slope       303 non-null int64
ca          303 non-null object
thal        303 non-null object
class       303 non-null int64
dtypes: float64(1), int64(11), object(2)
memory usage: 33.2+ KB


In [11]:
cat_dummies = pd.get_dummies(patients['class'])
patients = pd.concat([patients, cat_dummies], axis=1)
#del patients['class']

In [12]:
patients.head()

,age,gender,cp,trestbps,chol,fps,restecg,thalach,exang,oldpeak,slope,ca,thal,class,0,1,2,3,4
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0,1,0,0,0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2,0,0,1,0,0
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1,0,1,0,0,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0,1,0,0,0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0,1,0,0,0,0


In [79]:
patients['class'].unique()

array([0, 2, 1, 3, 4], dtype=int64)

In [41]:
for i in ['ca','thal']:
    value = patients[i].value_counts().idxmax()
    patients[i] = patients[i].replace('?', value)
    patients[i] = patients[i].astype(int)


In [44]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 19 columns):
age         303 non-null int64
gender      303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
fps         303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
slope       303 non-null int64
ca          303 non-null int32
thal        303 non-null int32
class       303 non-null int64
0           303 non-null uint8
1           303 non-null uint8
2           303 non-null uint8
3           303 non-null uint8
4           303 non-null uint8
dtypes: float64(1), int32(2), int64(11), uint8(5)
memory usage: 32.3 KB


In [45]:
train = patients.iloc[:225]
test = patients.iloc[225:]

In [48]:
train.columns

Index([     'age',   'gender',       'cp', 'trestbps',     'chol',      'fps',
        'restecg',  'thalach',    'exang',  'oldpeak',    'slope',       'ca',
           'thal',    'class',          0,          1,          2,          3,
                4],
      dtype='object')

In [78]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

target = [0,1,2,3,4]
train_a = train.loc[:,'age':'thal']

features = {}
for i in target:
    logit = LogisticRegression()
    rfe = RFE(logit, 3)
    rfe = rfe.fit(train_a, train[i])
    features[i] = train_a.columns[rfe.support_]
    
features

C:\Users\Pranab\Miniconda3\envs\datascience\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Pranab\Miniconda3\envs\datascience\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Pranab\Miniconda3\envs\datascience\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Pranab\Miniconda3\envs\datascience\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Pranab\Miniconda3\envs\datascience\lib\site-packages\sklearn\linear_model\logistic.py:433: 

{0: Index(['exang', 'oldpeak', 'ca'], dtype='object'),
 1: Index(['gender', 'fps', 'exang'], dtype='object'),
 2: Index(['gender', 'exang', 'ca'], dtype='object'),
 3: Index(['exang', 'oldpeak', 'ca'], dtype='object'),
 4: Index(['cp', 'fps', 'ca'], dtype='object')}

In [75]:
print(train_a.columns[rfe.support_])
print(rfe.ranking_)

Index(['exang', 'oldpeak', 'ca'], dtype='object')
[ 6  8  5 11 10  3  7  9  1  1  2  1  4]
